# Data loading and  pre-processing

In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

def loaddata():
    dataset_mnist = np.load('mnist_dataset.npz')
    dataset_cifar = np.load('cifar_dataset.npz')

    Xtr_mnist = dataset_mnist['Xtr']
    Str_mnist = dataset_mnist['Str']
    Xts_mnist = dataset_mnist['Xts']
    Yts_mnist = dataset_mnist['Yts']

    Xtr_cifar = dataset_cifar['Xtr']
    Str_cifar = dataset_cifar['Str']
    Xts_cifar = dataset_cifar['Xts']
    Yts_cifar = dataset_cifar['Yts']

    return Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar


def standard(x_train, x_test):
    '''
    Note:
         Use the std and mean from the training data and implement
         it to the validation data and test data
    RETRUN:
        Flattened and standard(Z-score normalization) training data
        The features size is 3072 or 784
    '''
    std = np.std(x_train, keepdims=True)
    mean = np.mean(x_train, keepdims=True)
    x_train = (x_train-mean)/std
    x_test = (x_test-mean)/std
    return x_train, x_test


def split_data(x_train, y_train, part=0.8):
    all_index = np.arange(y_train.shape[0])
    np.random.shuffle(all_index)
    
    train_data_size = int(y_train.shape[0] * part)
    
    train_index = all_index[0:train_data_size]
    val_index= all_index[train_data_size:]
    
    
    x_vali = x_train[val_index]
    y_vali = y_train[val_index]

    x_train = x_train[train_index]
    y_train = y_train[train_index]

    return x_train, y_train, x_vali, y_vali,train_index


def one_hot_coding(y_train, y_test):
    '''
    Encode label to one-hot catogory
    '''
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)
    return y_train, y_test

def compute_acc(y_ture, y_pred):
    acc = np.sum(y_ture == np.argmax(y_pred, axis=1))/len(y_ture)
    return round(acc, 3)


# CNN model and estimate $\beta$

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import tensorflow as tf
import numpy as np


def train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                    lr=0.0001, decay=0.000001, batchsize=256, epochs=1):
    # data is mnist
    if x_train.shape[3] == 1:
        model = Sequential([
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu',input_shape=(28,28,1)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu'),
            Flatten(),
            Dense(1000, activation='relu', use_bias=True),
            Dense(500, activation='relu', use_bias=True),
            Dense(2, activation=tf.nn.softmax)
        ])
        
    # data is cifar
    else:
        model = Sequential([
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu',input_shape=(32,32,3)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu'),
            Flatten(),
            Dense(1000, activation='relu', use_bias=True),
            Dense(500, activation='relu', use_bias=True),
            Dense(2, activation=tf.nn.softmax)
        ])

    adm = optimizers.Adam(lr=lr, decay=decay)
    if TYPE == 'with_noise':
        model.compile(loss='categorical_crossentropy',
                      optimizer=adm,
                      metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batchsize, \
                        epochs=epochs)
    elif TYPE == 'without_noise':
        model.compile(loss='categorical_crossentropy',
                      optimizer=adm,
                      metrics=['accuracy'])
        
        history = model.fit(x_train, y_train, batch_size=batchsize,sample_weight = weights.flatten(), \
                        epochs=epochs)
    return model, history


def estimateBeta(S,prob,rho0,rho1):
    n = len(S)
    beta = np.zeros((n,1))
    for i in range(n):
        if S[i]==1:
            beta[i] = (prob[i][1]-rho0)/((1-rho0-rho1)*prob[i][1]+1e-5)
        else:
            beta[i] = (prob[i][0]-rho1)/((1-rho0-rho1)*(prob[i][0])+1e-5)

    return beta


# Load data

In [3]:
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()
Xtr_mnist,Xts_mnist = standard(Xtr_mnist,Xts_mnist)
Xtr_mnist = Xtr_mnist.reshape(Xtr_mnist.shape[0], 28, 28,1)
Xts_mnist = Xts_mnist.reshape(Xts_mnist.shape[0], 28, 28,1)
y_train_o, y_test = one_hot_coding(Str_mnist,Yts_mnist)

print('x_train {}, y_train {} \nXts_mnist {}, y_test {}'.format(
    Xtr_mnist.shape,y_train_o.shape,Xts_mnist.shape,y_test.shape))

rho0 = 0.2
rho1 = 0.4

x_train (10000, 28, 28, 1), y_train (10000, 2) 
Xts_mnist (2000, 28, 28, 1), y_test (2000, 2)


# Training without importance reweighting for Fashion-MNIST
Use the parameters we got from cross validation step   
Randomly select 8000 samples from training set, use these to train models   
Calculate the probabilities which would be used to calculate weight $\beta$

In [4]:
for_flip_rate_1_l = []
for_flip_rate_0_l = []
acc_l = []
run_times = 10
x_train_x_results = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_mnist,y_train_o,part=0.8)
    model_noise,history_noise = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                        lr=0.00001, decay=0.000001, batchsize=1024, epochs=10)
    
    train_x_result = model_noise.predict(Xtr_mnist)

    for_flip_rate_1_l.append(np.min(train_x_result[:,0]))
    for_flip_rate_0_l.append(np.min(train_x_result[:,1]))

    l,a=model_noise.evaluate(Xts_mnist,y_test)
    
    acc_l.append(a)
    x_train_x_results.append(train_x_result)

Epoch 1/10
8000/8000 [==============================] - 2s 206us/step - loss: 0.6784 - acc: 0.5639
Epoch 2/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6539 - acc: 0.6102
Epoch 3/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6387 - acc: 0.6405
Epoch 4/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6273 - acc: 0.6410
Epoch 5/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6187 - acc: 0.6420
Epoch 6/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6115 - acc: 0.6571
Epoch 7/10
8000/8000 [==============================] - 0s 34us/step - loss: 0.6062 - acc: 0.6619
Epoch 8/10
8000/8000 [==============================] - 0s 33us/step - loss: 0.6024 - acc: 0.6735
Epoch 9/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.5993 - acc: 0.6738
Epoch 10/10
2000/2000 [==============================] - 0s 56us/step
Epoch 1/10
8000/8000 [=========================

2000/2000 [==============================] - 0s 105us/step
Epoch 1/10
8000/8000 [==============================] - 1s 118us/step - loss: 0.6819 - acc: 0.5494
Epoch 2/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6548 - acc: 0.6036
Epoch 3/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6393 - acc: 0.6282
Epoch 4/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6302 - acc: 0.6373
Epoch 5/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6217 - acc: 0.6392
Epoch 6/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6154 - acc: 0.6549
Epoch 7/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6109 - acc: 0.6537
Epoch 8/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6074 - acc: 0.6640
Epoch 9/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6049 - acc: 0.6663
Epoch 10/10
2000/2000 [==============================] - 0

In [5]:
# print(for_flip_rate_0_l)
# print(for_flip_rate_1_l)

# print(acc_l)

print(sum(acc_l)/len(acc_l))
print(np.array(acc_l).std())

0.8436
0.026624988262908204


# Average probabilities and calculate weight $\beta$

In [6]:
for i in range(len(x_train_x_results)):
    if i == 0:
        total_x_result = np.copy(x_train_x_results[i])
    else:
        total_x_result += x_train_x_results[i]
train_x_result = total_x_result/(run_times)


all_weights = estimateBeta(Str_mnist, train_x_result, rho0, rho1)
for i in range(len(all_weights)):
    if all_weights[i] < 0:
        all_weights[i] = 0.0

# Training with importance reweighting method for Fashion-MNIST
Randomly select 8000 samples from training set, use these to train models   
Calculate accuracy score

In [7]:
acc_l_without = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_mnist,y_train_o,part=0.8)
    
    weights = all_weights[split_index]
    
    model, history = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='without_noise',
                        lr=0.00001, decay=0.000001, batchsize=1024, epochs=10)
    
#     model.evaluate(Xts_mnist,y_test)
    l,a=model.evaluate(Xts_mnist,y_test)
    acc_l_without.append(a)


Epoch 1/10
8000/8000 [==============================] - 1s 125us/step - loss: 0.6643 - acc: 0.5864
Epoch 2/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.6036 - acc: 0.6281
Epoch 3/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.5529 - acc: 0.6450
Epoch 4/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.5076 - acc: 0.6526
Epoch 5/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.4684 - acc: 0.6577
Epoch 6/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.4345 - acc: 0.6630
Epoch 7/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.4064 - acc: 0.6650
Epoch 8/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.3838 - acc: 0.6679
Epoch 9/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.3665 - acc: 0.6709
Epoch 10/10
2000/2000 [==============================] - 0s 111us/step
Epoch 1/10
8000/8000 [========================

2000/2000 [==============================] - 0s 174us/step
Epoch 1/10
8000/8000 [==============================] - 1s 156us/step - loss: 0.6640 - acc: 0.5982
Epoch 2/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.5855 - acc: 0.6304
Epoch 3/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.5282 - acc: 0.6466
Epoch 4/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.4806 - acc: 0.6595
Epoch 5/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.4405 - acc: 0.6641
Epoch 6/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.4070 - acc: 0.6676
Epoch 7/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.3807 - acc: 0.6690
Epoch 8/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.3612 - acc: 0.6682
Epoch 9/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.3468 - acc: 0.6681
Epoch 10/10
2000/2000 [==============================] - 0

# Calculate average accuracy and standard deviation

In [8]:
# print(acc_l_without)

print(sum(acc_l_without)/len(acc_l_without))
print(np.array(acc_l_without).std())

0.925
0.003981205847478865


# Cifar

In [4]:
Xtr_cifar,Xts_cifar = standard(Xtr_cifar,Xts_cifar)

Xtr_cifar = Xtr_cifar.reshape(Xtr_cifar.shape[0], 32,32,3)
Xts_cifar = Xts_cifar.reshape(Xts_cifar.shape[0], 32,32,3)

y_train_o, y_test = one_hot_coding(Str_cifar,Yts_cifar)

print('x_train {}, y_train {} \nXts_cifar {}, y_test {}'.format(
    Xtr_cifar.shape,y_train_o.shape,Xts_cifar.shape,y_test.shape))


x_train (10000, 32, 32, 3), y_train (10000, 2) 
Xts_cifar (2000, 32, 32, 3), y_test (2000, 2)


# Training without importance reweighting for Cifar
Use the parameters we got from cross validation step   
Randomly select 8000 samples from training set, use these to train models   
Calculate the probabilities which would be used to calculate weight $\beta$

In [5]:
for_flip_rate_1_l = []
for_flip_rate_0_l = []
acc_l = []
run_times = 10
x_train_x_results = []

for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_cifar,y_train_o,part=0.8)
    model_noise,history_noise = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                        lr=0.00005, decay=0.000001, batchsize=1024, epochs=10)
    
    train_x_result = model_noise.predict(Xtr_cifar)

    for_flip_rate_1_l.append(np.min(train_x_result[:,0]))
    for_flip_rate_0_l.append(np.min(train_x_result[:,1]))

    l,a=model_noise.evaluate(Xts_cifar,y_test)
    
    acc_l.append(a)
    x_train_x_results.append(train_x_result)

Epoch 1/10
8000/8000 [==============================] - 2s 247us/step - loss: 0.6855 - acc: 0.5765
Epoch 2/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.6444 - acc: 0.6241
Epoch 3/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.6350 - acc: 0.6396
Epoch 4/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.6207 - acc: 0.6659
Epoch 5/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.6119 - acc: 0.6707
Epoch 6/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.6050 - acc: 0.6824
Epoch 7/10
8000/8000 [==============================] - 0s 47us/step - loss: 0.6003 - acc: 0.6808
Epoch 8/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.5945 - acc: 0.6870
Epoch 9/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5838 - acc: 0.7009
Epoch 10/10
2000/2000 [==============================] - 0s 81us/step
Epoch 1/10
8000/8000 [=========================

2000/2000 [==============================] - 0s 125us/step
Epoch 1/10
8000/8000 [==============================] - 1s 144us/step - loss: 0.6527 - acc: 0.6190
Epoch 2/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.6257 - acc: 0.6571
Epoch 3/10
8000/8000 [==============================] - 0s 49us/step - loss: 0.6133 - acc: 0.6774
Epoch 4/10
8000/8000 [==============================] - 0s 49us/step - loss: 0.6011 - acc: 0.6840
Epoch 5/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.5905 - acc: 0.7003
Epoch 6/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.5795 - acc: 0.7114
Epoch 7/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5738 - acc: 0.7179
Epoch 8/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5644 - acc: 0.7246
Epoch 9/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.5468 - acc: 0.7388
Epoch 10/10
2000/2000 [==============================] - 0

In [6]:
# print(for_flip_rate_0_l)
# print(for_flip_rate_1_l)

# print(acc_l)

print(sum(acc_l)/len(acc_l))
print(np.array(acc_l).std())

0.8109
0.04070245692829858


# Average probabilities and calculate weight $\beta$

In [8]:
for i in range(len(x_train_x_results)):
    if i == 0:
        total_x_result = np.copy(x_train_x_results[i])
    else:
        total_x_result += x_train_x_results[i]
        
train_x_result = total_x_result/(run_times)

all_weights = estimateBeta(Str_cifar, train_x_result, rho0, rho1)
for i in range(len(all_weights)):
    if all_weights[i] < 0:
        all_weights[i] = 0.0

# Training with importance reweighting method for Cifar
Randomly select 8000 samples from training set, use these to train models   
Calculate accuracy score

In [9]:
acc_l_without = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_cifar,y_train_o,part=0.8)
    
    weights = all_weights[split_index]
    
    model, history = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='without_noise',
                        lr=0.00005, decay=0.000001, batchsize=1024, epochs=10)

    l,a=model.evaluate(Xts_cifar,y_test)
    acc_l_without.append(a)

Epoch 1/10
8000/8000 [==============================] - 1s 153us/step - loss: 0.5953 - acc: 0.5703
Epoch 2/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.4995 - acc: 0.6206
Epoch 3/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.4642 - acc: 0.6323
Epoch 4/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.4354 - acc: 0.6438
Epoch 5/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.4169 - acc: 0.6555
Epoch 6/10
8000/8000 [==============================] - 0s 47us/step - loss: 0.4000 - acc: 0.6594
Epoch 7/10
8000/8000 [==============================] - 0s 50us/step - loss: 0.3852 - acc: 0.6694
Epoch 8/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.3740 - acc: 0.6771
Epoch 9/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.3626 - acc: 0.6818
Epoch 10/10
2000/2000 [==============================] - 0s 134us/step
Epoch 1/10
8000/8000 [========================

2000/2000 [==============================] - 0s 196us/step
Epoch 1/10
8000/8000 [==============================] - 1s 180us/step - loss: 0.5847 - acc: 0.5649
Epoch 2/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.4777 - acc: 0.6250
Epoch 3/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.4501 - acc: 0.6354
Epoch 4/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.4268 - acc: 0.6506
Epoch 5/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.4107 - acc: 0.6549
Epoch 6/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.3981 - acc: 0.6654
Epoch 7/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.3853 - acc: 0.6669
Epoch 8/10
8000/8000 [==============================] - 0s 44us/step - loss: 0.3739 - acc: 0.6723
Epoch 9/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.3663 - acc: 0.6778
Epoch 10/10
2000/2000 [==============================] - 0

# Calculate average accuracy and standard deviation

In [10]:
# print(acc_l_without)

print(sum(acc_l_without)/len(acc_l_without))
print(np.array(acc_l_without).std())

0.8789
0.005629387178015032
